## Classification Model

In [88]:
from keras.layers import Dense, Flatten, Conv2D, MaxPooling2D, BatchNormalization, Input
from keras.models import Model
from keras.utils import to_categorical
from sklearn.preprocessing import OneHotEncoder, LabelEncoder

imgs = np.load('75/images.npy')
imgs = imgs.astype('float32')
split = int(18000*0.7)
train_imgs = imgs[100:split]
test_imgs = imgs[split:]
val_imgs = imgs[0:100]

labels = np.load('75/labels.npy')
labels = labels.astype('int32')
train_labels = labels[100:split]
test_labels = labels[split:]
val_labels = labels[0:100]

def conv_time(time):
    # ntime = 0
    # if time[1] > 30:
    #     ntime = time[0]
    # else:
    #     ntime = time[0] + 0.5
    return time[0]

train_labels = np.array([conv_time(time) for time in train_labels])
test_labels = np.array([conv_time(time) for time in test_labels])
val_labels = np.array([conv_time(time) for time in val_labels])

encoder = LabelEncoder()
test_labels_encoded = encoder.fit_transform(test_labels.reshape(-1))
train_labels_encoded = encoder.fit_transform(train_labels.reshape(-1))
val_labels_encoded = encoder.fit_transform(val_labels.reshape(-1))

encoder = OneHotEncoder(sparse_output=False)
train_labels = encoder.fit_transform(train_labels_encoded.reshape(-1, 1))
val_labels = encoder.fit_transform(val_labels_encoded.reshape(-1, 1))


print(train_imgs.shape, train_labels.shape)

input_shape = (75, 75, 1)
model = keras.models.Sequential()
model.add(keras.layers.Conv2D(kernel_size=(5,5), strides = (2,2), activation="relu", filters=32))
model.add(keras.layers.Conv2D(activation="relu", filters=32, kernel_size=(3,3), input_shape=input_shape))
model.add(keras.layers.MaxPooling2D(pool_size=2))
model.add(keras.layers.Conv2D(kernel_size=(3,3), activation="relu", filters=32))
model.add(keras.layers.Conv2D(kernel_size=(3,3), activation="relu", filters=32))
model.add(keras.layers.MaxPooling2D(pool_size=2))
model.add(keras.layers.Conv2D(kernel_size=(3,3), activation="relu", filters=64))
model.add(keras.layers.Conv2D(kernel_size=(3,3), activation="relu", filters=64))

model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(units=512, activation="relu"))
model.add(keras.layers.Dense(units=256, activation="relu"))
model.add(keras.layers.Dense(units=128, activation="relu"))
model.add(keras.layers.Dense(units=64, activation="relu"))
model.add(keras.layers.Dense(units=32, activation="relu"))
model.add(keras.layers.Dense(units=17, activation="softmax"))

optimizer = keras.optimizers.Adam(learning_rate=0.001)

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.fit(tf.expand_dims(train_imgs, axis=-1), train_labels, epochs=10, batch_size=32, validation_data=(tf.expand_dims(val_imgs, axis=-1), val_labels))
model.summary()
     

(12500, 75, 75) (12500, 17)
Epoch 1/10
391/391 [==============================] - ETA: 0s - loss: 2.8336 - accuracy: 0.0593

ValueError: in user code:

    File "c:\Users\rajat\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\engine\training.py", line 2066, in test_function  *
        return step_function(self, iterator)
    File "c:\Users\rajat\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\engine\training.py", line 2049, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "c:\Users\rajat\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\engine\training.py", line 2037, in run_step  **
        outputs = model.test_step(data)
    File "c:\Users\rajat\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\engine\training.py", line 1919, in test_step
        self.compute_loss(x, y, y_pred, sample_weight)
    File "c:\Users\rajat\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\engine\training.py", line 1209, in compute_loss
        return self.compiled_loss(
    File "c:\Us

In [46]:
import numpy as np
import matplotlib.pyplot as plt
import keras
import tensorflow as tf

In [ ]:
def predict(img):
    return model.predict(img.reshape(1, 75, 75, 1))